# Input file definitions: Edit this as data is updated

In [19]:
# this is where we define the input excel

# final_phase_1_excel = 'Phase 1 Dashboard PES 06022023.xlsx'
final_phase_1_excel = "Completed Dashboard March 2022 Peter's version + KP.xlsx"
phase2_excel = '230208 Phase 2 December_Final for Open Up.xlsx'


# For updates, don't edit below this point

In [20]:
import functools
import json
import math
import operator
import re
import sys
from datetime import datetime

import numpy as np
import pandas as pd

from pprint import PrettyPrinter
sys.path.append("../python-src")
from presidential_employment import *

In [21]:
pp = PrettyPrinter(indent=2)

In [22]:
output_dir = "/home/pvh/Documents/code/pvh-forks/presidential-employment-stimulus/data"

### Data structure

Each department has a total budget and total opportunities target. 

The overall programme has outcome targets
1. Jobs created
2. Jobs retained
3. Livelihoods supported

Each department has a "blurb" describing their programme.

Within each department there are multiple sections programmes that can contribute to each of these targets.

Each programme has a demographic split of outcomes, with gender and youth percentages.

Each programme has a per-province split of outcomes.

Files:

`Phase 1 Dashboard input_PES targets and opportunities per month 09022022 Updated with Implementation 2.xlsx` - final phase 1 Excel
`April dashboard_2.xlsx` - latest phase 2 Excel


In [23]:
# dump metric titles (defined in python_src/presidential_employment.py) into metric_title.json
json.dump(metric_titles, open(output_dir + "/metric_titles.json", "w"), indent=2)

In [24]:
(opportunity_targets_df,
 opportunity_achievements_df,
 implementation_status_df,
 description_df,
 phase1_departments,
 phase2_departments,
 targets_df,
 trends_df,
 provincial_df,
 cities_df,
 universities_df,
 demographic_df,
 achievement_totals_df) = load_sheets(final_phase_1_excel, phase2_excel)

In [25]:
department_names = list(set(phase1_departments).union(phase2_departments))

leads = description_df.lead.to_dict()
paragraphs = description_df.paragraph.to_dict()

In [26]:
pd.set_option('display.max_rows', None)
targets_df[0]

,department,programme,target,section,display_name
0,Basic Education,Basic Education Employment Initiative Phase I,300000.0,CRE,PYEI-BEEI Phase I
1,Basic Education,Basic Education Employment Initiative Phase II,287000.0,CRE,PYEI-BEEI Phase II
2,Basic Education,Retain vulnerable teaching posts,44933.0,RET,Retain vulnerable teaching posts
3,Social Development,ECD income and compliance support,151551.0,LIV,ECD income and compliance support
4,Social Development,Registration support officers,500.0,CRE,Registration support officers
5,Social Development,Retention of social workers,3809.0,RET,Retention of social workers
6,"Agriculture, Land Reform and Rural Development",Subsistence producer relief fund,74626.0,LIV,Subsistence producer relief fund
7,"Agriculture, Land Reform and Rural Development",Vegetables and Fruits,NaN,LIV,Vegetables and Fruits
8,"Agriculture, Land Reform and Rural Development",Maize/soya/sugar/other production,NaN,LIV,Maize/soya/sugar/other production
9,"Agriculture, Land Reform and Rural Development",Poultry: Layers and Boilers,NaN,LIV,Poultry: Layers and Boilers


In [27]:
targets_df[0][targets_df[0].section == 'CRE'].target.sum()

1791023.0

In [28]:
targets_df[1]

,department,programme,target,unk,section,display_name
0,Basic Education,Education Assistants,191392.0,NaN,CRE,Education Assistants
1,Basic Education,General Assistants,95608.0,NaN,CRE,General Assistants
2,Social Development,ECD Employment Stimulus: Complete payments ove...,42718.0,NaN,LIV,ECD Employment Stimulus: Complete payments ove...
3,Social Development,Appointment of social workers to assist SASSA ...,2000.0,NaN,CRE,Appointment of social workers to assist SASSA ...
4,Social Development,NDA Volunteer Programme,1880.0,NaN,CRE,NDA Volunteer Programme
5,"Agriculture, Land Reform and Rural Development",Subsistence Producer Relief Fund,67378.0,HAS_SP,LIV,Subsistence producer relief fund
6,"Agriculture, Land Reform and Rural Development",Vegetables and Fruits,NaN,NaN,LIV,Vegetables and Fruits
7,"Agriculture, Land Reform and Rural Development",Maize/soya/sugar/other production,NaN,NaN,LIV,Maize/soya/sugar/other production
8,"Agriculture, Land Reform and Rural Development",Poultry: Layers and Boilers,NaN,NaN,LIV,Poultry: Layers and Boilers
9,"Agriculture, Land Reform and Rural Development",Small livestock,NaN,NaN,LIV,Small livestock


## 

In [29]:
def add_or_replace(departments, department):
    # if a department with sheet_name exists in the list, replace it with the new department, else append to list
    for i, el in enumerate(departments):
        if el.sheet_name == department.sheet_name:
            departments[i] = department
            break
    else:
        departments.append(department)
    return departments

## Compute per department data structures

In [30]:
all_data_departments = compute_all_data_departments(phase1_departments, phase2_departments, 
                                                    implementation_status_df, demographic_df, description_df,
                                                    targets_df, trends_df, department_names, provincial_df,
                                                    cities_df, universities_df, leads, paragraphs)

/home/pvh/Documents/code/pvh-forks/presidential-employment-stimulus/notebooks/../python-src/presidential_employment.py:925: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  value = int(row.loc[:, key])


ACHIEVED: Retain vulnerable teaching posts 31114 44933.0
Implementation status missing for:  0 : Public Works and Infrastructure : Water and Energy Efficiency
Implementation status missing for:  0 : Public Works and Infrastructure : Water and Sanitation Facilities Management
Implementation status missing for:  0 : Public Works and Infrastructure : Welisizwe Rural Bridges Programme
Implementation status missing for:  0 : Public Works and Infrastructure : Facilities Management
Implementation status missing for:  0 : Public Works and Infrastructure : Real Estate
Implementation status missing for:  0 : Public Works and Infrastructure : In-House Construction projects
Implementation status missing for:  0 : Public Works and Infrastructure : Public Private Collaborations
Implementation status missing for:  0 : Agriculture, Land Reform and Rural Development : Graduate verifiers
Implementation status missing for:  0 : Agriculture, Land Reform and Rural Development : Vegetables and Fruits
Implem

## Compute breakdown of all programmes by demographic dimensions

In [31]:
(total_male, 
 total_female, 
 total_unknown_gender,
 total_beneficiaries,
 total_youth, 
 total_unknown_youth, 
 total_provincial, 
 total_unknown_province) = compute_breakdowns(all_data_departments)

## Overview picture

In [32]:
(programmes_by_type,
 programmes_by_type_summarised,
 achievements_by_type_by_month,
 provincial_breakdown) = compute_programmes_by_type(all_data_departments, opportunity_achievements_df, opportunity_targets_df)

### Check that total add up to totals listed in the spreadsheet

In [33]:
# check targets for phase 1 - job opportunities
assert (
    programmes_by_type[SectionEnum.job_opportunities.name][0]["Total"]["value_target"]
    == opportunity_targets_df[0].iloc[6, 7]
), f'{SectionEnum.job_opportunities.name} total mismatch: {programmes_by_type[SectionEnum.job_opportunities.name][0]["Total"]["value_target"]} vs {opportunity_targets_df[0].iloc[6, 7]}'

# check targets for phase 2 - job opportunities
assert (
    programmes_by_type[SectionEnum.job_opportunities.name][1]["Total"]["value_target"]
    == opportunity_targets_df[1].iloc[5, 7]
), f'{SectionEnum.job_opportunities.name} total mismatch: {programmes_by_type[SectionEnum.job_opportunities.name][1]["Total"]["value_target"]} vs {opportunity_targets_df[1].iloc[5, 7]}'


In [34]:
# check achievements for phase 1 - job opportunities
assert (
    programmes_by_type[SectionEnum.job_opportunities.name][0]["Total"]["value"] == achievement_totals_df[0].loc["Jobs created","total"]
), f'Phase 1: {SectionEnum.job_opportunities.name} total mismatch computed {programmes_by_type[SectionEnum.job_opportunities.name][0]["Total"]["value"]} vs sheet {achievement_totals_df[0].loc["Jobs created"]}'

# check achiements for phase 2 - job opportunities
assert (
    programmes_by_type[SectionEnum.job_opportunities.name][1]["Total"]["value"] == achievement_totals_df[1].loc["Jobs created","total"]
), f'Phase 2: {SectionEnum.job_opportunities.name} total mismatch {programmes_by_type[SectionEnum.job_opportunities.name][1]["Total"]["value"]} vs {achievement_totals_df[1].loc["Jobs created"]}'


In [35]:
# check targets for phase 1 - livelihoods support
assert (
    programmes_by_type[SectionEnum.livelihoods.name][0]["Total"]["value_target"]
    == opportunity_targets_df[0].iloc[7, 7]
), f'{SectionEnum.livelihoods.name} total mismatch: phase 1 target {programmes_by_type[SectionEnum.livelihoods.name][0]["Total"]["value_target"]} vs {opportunity_targets_df[0].iloc[7, 7]}'

# check targets for phase 2 - livelihoods support
assert (
    programmes_by_type[SectionEnum.livelihoods.name][1]["Total"]["value_target"]
    == opportunity_targets_df[1].iloc[6, 7]
), f'{SectionEnum.livelihoods.name} total mismatch: phase 2 target {programmes_by_type[SectionEnum.livelihoods.name][1]["Total"]["value_target"]} vs {opportunity_targets_df[1].iloc[6, 7]}'


# check achievements for phase 1 - livelihoods support
assert (
    programmes_by_type[SectionEnum.livelihoods.name][0]["Total"]["value"] == achievement_totals_df[0].loc["Livelihoods supported","total"]
), f'{SectionEnum.job_opportunities.name} total mismatch - phase 1 {programmes_by_type[SectionEnum.livelihoods.name][0]["Total"]["value"]} vs {achievement_totals_df[0].loc["Livelihoods supported"]}'

# check achievements for phase 2 - livelihoods support
assert (
    programmes_by_type[SectionEnum.livelihoods.name][1]["Total"]["value"] == achievement_totals_df[1].loc["Livelihoods supported","total"]
), f'{SectionEnum.job_opportunities.name} total mismatch - phase 2 {programmes_by_type[SectionEnum.livelihoods.name][1]["Total"]["value"]} vs {achievement_totals_df[1].loc["Livelihoods supported"]}'


In [36]:
# check targets for phase 1 - jobs retained
assert (
    programmes_by_type[SectionEnum.jobs_retain.name][0]["Total"]["value_target"]
    == opportunity_targets_df[0].iloc[8, 7]
), f'{SectionEnum.jobs_retain.name} total mismatch: {programmes_by_type[SectionEnum.jobs_retain.name][0]["Total"]["value_target"]} vs {opportunity_targets_df[0].iloc[8, 7]}'

# check achivements for phase 2 - jobs retained
assert (
    programmes_by_type[SectionEnum.jobs_retain.name][0]["Total"]["value"] == achievement_totals_df[0].loc["Jobs retained","total"]
), f'{SectionEnum.job_opportunities.name} total mismatch {programmes_by_type[SectionEnum.jobs_retain.name][0]["Total"]["value"]} vs {achievement_totals_df[0].loc["Jobs retained"]}'


### Compute breakdowns used in overview and metrics used in overview

In [37]:
(breakdown_metrics, current_targets, current_achievements) = compute_overview_breakdown(programmes_by_type_summarised,
                                                                                        achievements_by_type_by_month,
                                                                                        provincial_breakdown)

overview_metrics = compute_overview_metrics(total_female, total_beneficiaries, total_unknown_gender,
                             opportunity_targets_df, programmes_by_type,
                             total_youth, total_unknown_youth)


### Assemble overview and put together final combined data

In [38]:
overview = compute_overview(description_df, leads, paragraphs, overview_metrics,
                            current_targets, current_achievements, breakdown_metrics)
all_data = Everything(
    overview=overview,
    departments=all_data_departments
)


In [39]:
# ad_set = set()
# for department in all_data.departments:
#     for section in department.sections:
#         if section.section_type == SectionEnum.targets.name:
#             continue
#         for metric in section.metrics:
#             ad_set.add(metric.name)

# ot_set = set(list(opportunity_targets_df.iloc[2:55, 1]))
# imp_set = set(list(implementation_status_df.iloc[:53, 1]))
# ot_set.difference(ad_set)

In [40]:
# for department in all_data.departments:
#     print(department.name)
#     for section in department.phases[1].sections:
#         print(section.name)
#         for metric in section.metrics:
#             print(metric.name)
#             metric.to_json()
# import inspect
# print("mod", inspect.getmodule(type(all_data.departments[0].phases[1].sections[0].metrics[1].value_target)))
# inspect.getmodule(type(all_data.departments[0].phases[1].sections[0].metrics[1].value_target)) == np

# Save final data

In [41]:
# to work around the fact that data from pandas sometimes appears as numpy types, this uses a
# version of dataclasses-json core.py (https://github.com/pvanheus/dataclasses-json/blob/master/dataclasses_json/core.py)
# see this PR: https://github.com/lidatong/dataclasses-json/pull/352
output_filename = output_dir + "/all_data.json"
# all_data.departments.sort(key=operator.attrgetter("sheet_name"))
open(output_filename, "w").write(all_data.to_json(indent=2))
# print(all_data.to_json(indent=2))
print("DONE")

DONE


In [42]:
# programme_status_df = pd.read_excel(
#     mar_opportunities_excel, sheet_name="Implementation status", header=None
# )

# to_camel_case = lambda match: match.group(1) + match.group(2).upper() + match.group(3)

# [
#     re.sub(r"(\S*) (\w)(.*)", to_camel_case, status)
#     for status in implementation_status_df.iloc[3:, 2].dropna().unique()
# ]

In [43]:
# for dept in all_data.departments:
#     print(f"\t'{dept.name}': '{dept.sheet_name}',")

In [44]:
# json.dump(metric_titles, open(output_dir + "/metric_titles.json", "w"), indent=2)

In [45]:
implementation_status_df[1]

,department,programme,status,detail
0,Basic Education,Education Assistants,On track,Most of the second cohort of school assistants...
1,Basic Education,General Assistants,On track,Much effort has gone into supporting the schoo...
2,Social Development,ECD Employment Stimulus: Complete payments ove...,Critical challenges,Additional payments made since October 2021 we...
3,Social Development,Appointment of social workers to assist SASSA ...,On track,The programme exceeded its target.
4,Social Development,NDA Volunteer Programme,On track,The NDA is partnering with Civil Society Organ...
5,"Agriculture, Land Reform and Rural Development",Subsistence Producer Relief Fund,On track,"The beneficiaries under phase 2 includes 51,55..."
6,"Agriculture, Land Reform and Rural Development",Vegetables and Fruits,NaN,
7,"Agriculture, Land Reform and Rural Development",Maize/soya/sugar/other production,NaN,
8,"Agriculture, Land Reform and Rural Development",Poultry: Layers and Boilers,NaN,
9,"Agriculture, Land Reform and Rural Development",Small livestock,NaN,
